In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString                  as B
import qualified Data.ByteString.Char8            as B (lines)
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as AC

input <- B.lines <$> B.readFile "../input/day12.txt"

In [2]:
data Register = A | B | C | D deriving (Eq, Show, Ord)

data Instruction
    = Cpy (Either Int Register) Register
    | Inc Register
    | Dec Register
    | Jnz (Either Int Register) Int
    deriving (Eq, Show)

parseRegister = A.choice
    [ A.string "a" *> (return A)
    , A.string "b" *> (return B)
    , A.string "c" *> (return C)
    , A.string "d" *> (return D)
    ]

parseCpy = do
    A.string "cpy"
    AC.space
    value <- A.choice [Left <$> AC.decimal, Right <$> parseRegister]
    AC.space
    reg <- parseRegister
    return $ Cpy value reg

parseInc = do
    A.string "inc"
    AC.space
    reg <- parseRegister
    return $ Inc reg

parseDec = do
    A.string "dec"
    AC.space
    reg <- parseRegister
    return $ Dec reg

parseJnz = do
    A.string "jnz"
    AC.space
    reg <- A.choice [Left <$> AC.decimal, Right <$> parseRegister]
    AC.space
    jmp <- AC.signed AC.decimal
    return $ Jnz reg jmp

parseInstruction = A.choice [parseCpy, parseInc, parseDec, parseJnz]

parsed = map (either error id . A.parseOnly parseInstruction) input
parsed

[Cpy (Left 1) A,Cpy (Left 1) B,Cpy (Left 26) D,Jnz (Right C) 2,Jnz (Left 1) 5,Cpy (Left 7) C,Inc D,Dec C,Jnz (Right C) (-2),Cpy (Right A) C,Inc A,Dec B,Jnz (Right B) (-2),Cpy (Right C) B,Dec D,Jnz (Right D) (-6),Cpy (Left 13) C,Cpy (Left 14) D,Inc A,Dec D,Jnz (Right D) (-2),Dec C,Jnz (Right C) (-5)]

In [3]:
-- import Control.Monad.Trans.State
import qualified Data.Map.Strict as M

type Registers = M.Map Register Int
data Computer = Computer
    { pointer      :: Int
    , registers    :: Registers
    } deriving (Eq, Show)

runInstr (Computer p rs) (Cpy (Left value) reg) = let
    rs' = M.insert reg value rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Cpy (Right reg1) reg2) = let
    value = rs M.! reg1
    rs'   = M.insert reg2 value rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Inc reg) = let
    value = rs M.! reg
    rs'   = M.insert reg (value+1) rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Dec reg) = let
    value = rs M.! reg
    rs'   = M.insert reg (value-1) rs
    in Computer (p+1) rs'
runInstr (Computer p rs) (Jnz (Left value) jmp) = if value /=0
    then Computer (p+jmp) rs
    else Computer (p+1)   rs
runInstr (Computer p rs) (Jnz (Right reg) jmp) = let
    value = rs M.! reg
    in if value /=0
        then Computer (p+jmp) rs
        else Computer (p+1)   rs
    
run c@(Computer p rs) instructions 
    | p >= length instructions = c
    | otherwise = let
        i  = instructions !! p
        c' = runInstr c i
        in run c' instructions

start = Computer 0 $ M.fromList [(A, 0), (B, 0), (C, 0), (D, 0)]

run start parsed

Computer {pointer = 23, registers = fromList [(A,317993),(B,196418),(C,0),(D,0)]}

In [4]:
start' = Computer 0 $ M.fromList [(A, 0), (B, 0), (C, 1), (D, 0)]
run start' parsed

Computer {pointer = 23, registers = fromList [(A,9227647),(B,5702887),(C,0),(D,0)]}